In [1]:
import pandas as pd
import numpy as np

In [2]:
all_user = pd.read_csv("data_identification.csv")
tweets = pd.read_csv("tweet_.csv", lineterminator='\n')
tweets = tweets.drop(columns=['Unnamed: 0'], axis=1)
print(all_user.shape)
print(tweets.shape)
tweets.head()

all_user = pd.merge(all_user,tweets, on="tweet_id")
all_user.head()

(1867535, 2)
(1867535, 4)


,tweet_id,identification,_score,hashtags,text
0,0x28cc61,test,107,[],@Habbo I've seen two separate colours of the e...
1,0x29e452,train,809,[],Huge Respect🖒 @JohnnyVegasReal talking about l...
2,0x2b3819,train,808,"['spateradio', 'app']",Yoooo we hit all our monthly goals with the ne...
3,0x2db41f,test,728,[],@FoxNews @KellyannePolls No serious self respe...
4,0x2a2acc,train,16,[],@KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...


In [3]:
fliter = (all_user["identification"] == 'train')
df_train = all_user[fliter]
fliter1 = (all_user["identification"] == 'test')
df_test = all_user[fliter1]
print(df_train.head(), df_train.shape)
print(df_test.head(), df_test.shape)

   tweet_id identification  _score  \
1  0x29e452          train     809   
2  0x2b3819          train     808   
4  0x2a2acc          train      16   
5  0x2a8830          train     768   
6  0x20b21d          train      70   

                                            hashtags  \
1                                                 []   
2                              ['spateradio', 'app']   
4                                                 []   
5  ['PUBG', 'GamersUnite', 'twitch', 'BeHealthy',...   
6                       ['strength', 'bones', 'God']   

                                                text  
1  Huge Respect🖒 @JohnnyVegasReal talking about l...  
2  Yoooo we hit all our monthly goals with the ne...  
4  @KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...  
5  Come join @ambushman27 on #PUBG while he striv...  
6  @fanshixieen2014 Blessings!My #strength little...   (1455563, 5)
    tweet_id identification  _score            hashtags  \
0   0x28cc61           test     

In [4]:
emotion = pd.read_csv("emotion.csv")
df_train = pd.merge(df_train,emotion, on="tweet_id")
df_train.iloc[4:100]

df_train = df_train.drop(columns=['identification'],axis=1)
df_test = df_test.drop(columns=['identification'],axis=1)

In [5]:
import string

list_hash = df_train['hashtags'].tolist()
for i in range(len(list_hash)):
    if len(list_hash[i]) == 0:
        list_hash[i]=None
    else:
        list_hash[i]=list_hash[i][1:len(list_hash[i])-2]
        list_hash[i] = [word.strip(string.punctuation) for word in list_hash[i].split(", ")]
        list_hash[i] = ' '.join(list_hash[i])
df_train['hashtags'] = list_hash
df_train["Full_text"] = df_train["hashtags"] + " " + df_train["text"]
df_train.head()

list_hash = df_test['hashtags'].tolist()
for i in range(len(list_hash)):
    if len(list_hash[i]) == 0:
        list_hash[i]=None
    else:
        list_hash[i]=list_hash[i][1:len(list_hash[i])-2]
        list_hash[i] = [word.strip(string.punctuation) for word in list_hash[i].split(", ")]
        list_hash[i] = ' '.join(list_hash[i])
df_test['hashtags'] = list_hash
df_test["Full_text"] = df_test["hashtags"] + " " + df_test["text"]
df_test.head()


,tweet_id,_score,hashtags,text,Full_text
0,0x28cc61,107,,@Habbo I've seen two separate colours of the e...,@Habbo I've seen two separate colours of the ...
3,0x2db41f,728,,@FoxNews @KellyannePolls No serious self respe...,@FoxNews @KellyannePolls No serious self resp...
15,0x2466f6,491,womendrivers,"Looking for a new car, and it says 1 lady owne...","womendrivers Looking for a new car, and it say..."
23,0x23f9e9,28,robbingmembers,@cineworld “only the brave” just out and fount...,robbingmembers @cineworld “only the brave” jus...
31,0x1fb4e1,925,,Felt like total dog 💩 going into open gym and ...,Felt like total dog 💩 going into open gym and...


In [6]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)  

In [7]:
df_train['emotion'].unique()

array(['trust', 'joy', 'anticipation', 'fear', 'anger', 'sadness',
       'disgust', 'surprise'], dtype=object)

In [8]:
from transformers import *
from transformers import TFBertForSequenceClassification, BertTokenizer, TFBertModel, BertConfig
bert_tokenizer = BertTokenizer.from_pretrained("./bert-base-uncased/")
bert_model = TFBertForSequenceClassification.from_pretrained('./bert-base-uncased/',num_labels=8, from_pt=True, force_download=True)

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTrain

In [9]:
sent= 'how to train the model, lets look at how a trained model calculates its prediction.'
tokenized_sequence= bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =30,pad_to_max_length = True,
return_attention_mask = True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [10]:
import tensorflow as tf
df_train['gt'] = df_train['emotion'].map({'joy':0,'sadness':1, 'anticipation':2,'trust':3, 'surprise':4,'fear':5, 'disgust':6,'anger':7})
df_train.head()

,tweet_id,_score,hashtags,text,emotion,Full_text,gt
1220813,0x2fa8c1,665,thanks followers,41 I'd like to say #thanks to all my #follower...,trust,thanks followers 41 I'd like to say #thanks to...,3
1030835,0x370d18,146,carefree,You either accept me or you don't #carefree bu...,trust,carefree You either accept me or you don't #ca...,3
1126126,0x313fe2,128,blessed,When you been in a motorcycle accident flying ...,joy,blessed When you been in a motorcycle accident...,0
1042413,0x377d87,1016,Yael,Single radio s'ecoute a la campagne #Yael Naї...,anticipation,Yael Single radio s'ecoute a la campagne #Yae...,2
1021128,0x20d937,724,exciting IS151,Currently preparing for the Career Fair tomorr...,fear,exciting IS151 Currently preparing for the Car...,5


In [11]:
sentences=df_train['Full_text']
labels=df_train['gt']
len(sentences),len(labels)

(1455563, 1455563)

In [12]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

len(input_ids),len(attention_masks),len(labels)

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(1455563, 1455563, 1455563)

In [17]:
sentences_ans=df_test['Full_text']

input_ids_ans=[]
attention_masks_ans=[]

for sent in sentences_ans:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids_ans.append(bert_inp['input_ids'])
    attention_masks_ans.append(bert_inp['attention_mask'])

input_ids_ans=np.asarray(input_ids_ans)
attention_masks_ans=np.array(attention_masks_ans)

len(input_ids),len(attention_masks)

(1455563, 1455563)

In [13]:
from sklearn.model_selection import train_test_split
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(
    input_ids,labels,attention_masks,test_size=0.01)

In [16]:
train_inp

array([[  101,  1030,  2003, ...,     0,     0,     0],
       [  101,  1030,  2008, ...,     0,     0,     0],
       [  101,  3297,  2205, ...,     0,     0,     0],
       ...,
       [  101,  2069,  2028, ...,     0,     0,     0],
       [  101,  3712, 16558, ...,     0,     0,     0],
       [  101,  1999,  2793, ...,     0,     0,     0]])

In [14]:
print('\nBert Model',bert_model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  6152      
Total params: 109,488,392
Trainable params: 109,488,392
Non-trainable params: 0
_________________________________________________________________

Bert Model None


In [29]:
history=bert_model.fit([train_inp,train_mask],train_label,
                       batch_size=32,epochs=1,
                       validation_data=([val_inp,val_mask],val_label))

37552/45032 [========================>.....] - ETA: 4:06:48 - loss: 0.7508 - accuracy: 0.7277

KeyboardInterrupt: 

In [16]:
preds = bert_model.predict([val_inp,val_mask],batch_size=32)
pred_labels = preds.argmax(axis=1)
f1 = f1_score(val_label,pred_labels)
print('F1 score',f1)
print('Classification Report')
print(classification_report(val_label,pred_labels,target_names=target_names))

OSError: Unable to open file (unable to open file: name = './bert_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [30]:
preds = bert_model.predict([input_ids_ans, attention_masks_ans],batch_size=32)


In [31]:
pred_labels = preds.logits.argmax(axis=1)
df_test['emotion'] = pred_labels
df_test['ans'] = df_test['emotion'].map({0:'joy',1:'sadness', 2:'anticipation',3:'trust', 4:'surprise',5:'fear', 6:'disgust',7:'anger'})
df_test.head()

,tweet_id,_score,hashtags,text,Full_text,emotion,ans
0,0x28cc61,107,,@Habbo I've seen two separate colours of the e...,@Habbo I've seen two separate colours of the ...,4,surprise
3,0x2db41f,728,,@FoxNews @KellyannePolls No serious self respe...,@FoxNews @KellyannePolls No serious self resp...,1,sadness
15,0x2466f6,491,womendrivers,"Looking for a new car, and it says 1 lady owne...","womendrivers Looking for a new car, and it say...",6,disgust
23,0x23f9e9,28,robbingmembers,@cineworld “only the brave” just out and fount...,robbingmembers @cineworld “only the brave” jus...,1,sadness
31,0x1fb4e1,925,,Felt like total dog 💩 going into open gym and ...,Felt like total dog 💩 going into open gym and...,6,disgust


In [32]:
df_answer = pd.concat([df_test['tweet_id'],
                     df_test['ans']], axis=1)

df_answer = df_answer.rename(columns={'tweet_id':'id', 'ans':'emotion'})
df_answer.to_csv('ans.csv',encoding='utf-8', index=False)

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

tfIdfVectorizer_1000 = TfidfVectorizer(max_features = 6000, tokenizer=nltk.word_tokenize, use_idf=True)

tfIdfVectorizer_1000.fit(df_train['Full_text'])

t = tfIdfVectorizer_1000.transform(df_train['Full_text'])
t.shape

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(1455563, 3)

In [89]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

tfIdfVectorizer_1 = TfidfVectorizer(max_features = 5500, tokenizer=nltk.word_tokenize, use_idf=True, max_df=0.7)
tfIdfVectorizer_2 = TfidfVectorizer(max_features = 1000, tokenizer=nltk.word_tokenize, use_idf=True)

# tf_train_data = pd.concat([df_train['hashtags'], df_train['text']], axis=1)
# df_train['ti_hash'] = tfIdfVectorizer.fit_transform(df_train['hashtags']).toarray()
# df_train['ti_text'] = tfIdfVectorizer.fit_transform(df_train['text']).toarray()
tfIdfVectorizer_1.fit(df_train['text'])
tfIdfVectorizer_2.fit(df_train['hashtags'])

t1 = tfIdfVectorizer_1.transform(df_train['text'])
t2 = tfIdfVectorizer_2.transform(df_train['hashtags'])
print(t1.shape, t2.shape)
t = np.hstack((t1.toarray(),t2.toarray()))
t.shape

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(1455563, 5500) (1455563, 1000)


(1455563, 6500)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    t,
    df_train['emotion'], test_size=0.0001)

In [14]:
## deal with label (string -> one-hot)
import keras
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)



check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 1080989         sadness
1429083    anticipation
32358               joy
1010717         sadness
Name: emotion, dtype: object

y_train.shape:  (1455417,)
y_test.shape:  (146,)


## After convert
y_train[0:4]:
 [[0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]

y_train.shape:  (1455417, 8)
y_test.shape:  (146, 8)


In [15]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  3
output_shape:  8


In [16]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
# X_W0 = Dense(units=2048)(X)  # 64
# H0 = ReLU()(X_W0)

# X_W1 = Dense(units=1024)(H0)  # 64
# H1 = ReLU()(X_W1)

# # 2nd hidden layer
# H1_W2 = Dense(units=512)(H1)  # 64
# H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=256)(X)  # 4
H3 = ReLU()(H2_W3)

H3_W4 = Dense(units=128)(H3)  # 4
H4 = ReLU()(H3_W4)

H4_W5 = Dense(units=64)(H4)  # 4
H5 = ReLU()(H4_W5)

H5_W6 = Dense(units=output_shape)(H5)  # 4
H6 = Softmax()(H5_W6)

model_output = H6

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense (Dense)                (None, 256)               1024      
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 64)                0     

In [21]:
from keras.callbacks import CSVLogger

X_train = X_train.todense()
X_test = X_test.todense()


# training setting
epochs = 2
batch_size = 32

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_data = (X_test, y_test))
print('training finish')

Epoch 1/2
45482/45482 [==============================] - 118s 3ms/step - loss: 1.7894 - accuracy: 0.3545 - val_loss: 1.7944 - val_accuracy: 0.3562
Epoch 2/2
45482/45482 [==============================] - 118s 3ms/step - loss: 1.7886 - accuracy: 0.3545 - val_loss: 1.7930 - val_accuracy: 0.3562
training finish


In [83]:
## predict
X_ans = tfIdfVectorizer_1000.transform(df_test["Full_text"])


In [95]:
t1 = tfIdfVectorizer_1.transform(df_test['text'])
t2 = tfIdfVectorizer_2.transform(df_test['hashtags'])
X_ans = np.hstack((t1.toarray(),t2.toarray()))

In [96]:
# X_ans = X_ans.todense()
pred_result = model.predict(X_ans, batch_size=128)
pred_result[:5]

array([[0.02897748, 0.06151203, 0.18041576, 0.11653931, 0.235212  ,
        0.2121676 , 0.08424885, 0.08092703],
       [0.00225762, 0.00432948, 0.08080253, 0.00288493, 0.00121982,
        0.89825255, 0.00819059, 0.00206249],
       [0.03595785, 0.12556405, 0.18189023, 0.0252073 , 0.34570614,
        0.12542124, 0.0569565 , 0.10329662],
       [0.03118087, 0.04312646, 0.31728023, 0.06628045, 0.22604601,
        0.16803679, 0.07035042, 0.0776988 ],
       [0.03387901, 0.04004981, 0.23812151, 0.03013022, 0.31792313,
        0.20772758, 0.04795323, 0.08421544]], dtype=float32)

In [97]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['joy', 'sadness', 'joy', 'disgust', 'joy'], dtype=object)

In [98]:
df_test['emotion'] = pred_result
df_answer = pd.concat([df_test['tweet_id'],
                     df_test['emotion']], axis=1)
df_answer

,tweet_id,emotion
0,0x28cc61,joy
3,0x2db41f,sadness
15,0x2466f6,joy
23,0x23f9e9,disgust
31,0x1fb4e1,joy
...,...,...
1867495,0x2c4dc2,joy
1867496,0x31be7c,anticipation
1867500,0x1ca58e,sadness
1867515,0x35c8ba,joy


In [99]:
df_answer = df_answer.rename(columns={'tweet_id':'id'})
df_answer.to_csv('ans.csv',encoding='utf-8', index=False)